In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-25 12:52:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.01MB/s    in 0.2s    

2021-06-25 12:52:37 (6.01 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|   N|     

In [41]:
# Create new table with total votes greater than or equal to 20
tv = vine_df.filter("total_votes>=20")
tv.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|5          |32           |32         |N   |N                |
|R28BVHOEL6GCHZ|5          |21           |21         |N   |Y                |
|R27IB4BRKO71I0|2          |16           |20         |N   |Y                |
|RJ86OW12Y4NRL |5          |124          |128        |N   |Y                |
|R3KRABOWCH6ZE4|1          |48           |65         |N   |Y                |
|R1OAKARA30DIWX|4          |83           |89         |N   |Y                |
|R36LSV4WQ4VB1K|5          |17           |22         |N   |N                |
|R18BTOE5UGN7UA|5          |26           |28         |N   |Y                |
|R1DXLGDSK2WMLD|4          |22           |22         |N   |N                |
|RZG2OHQ12YYZI |5          |30           |31         |N   |Y    

In [44]:
# Create table with helpful votes filters by 50%
hv = tv.filter((tv.helpful_votes / tv.total_votes >= 0.5))
hv.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|5          |32           |32         |N   |N                |
|R28BVHOEL6GCHZ|5          |21           |21         |N   |Y                |
|R27IB4BRKO71I0|2          |16           |20         |N   |Y                |
|RJ86OW12Y4NRL |5          |124          |128        |N   |Y                |
|R3KRABOWCH6ZE4|1          |48           |65         |N   |Y                |
|R1OAKARA30DIWX|4          |83           |89         |N   |Y                |
|R36LSV4WQ4VB1K|5          |17           |22         |N   |N                |
|R18BTOE5UGN7UA|5          |26           |28         |N   |Y                |
|R1DXLGDSK2WMLD|4          |22           |22         |N   |N                |
|RZG2OHQ12YYZI |5          |30           |31         |N   |Y    

In [46]:
# Filter hv table for Vine program (paid)
vp_yes = hv.filter(hv.vine == 'Y')
vp_yes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R32OMS1LSDRCOB|          4|           27|         31|   Y|                N|
|R2IPKK4I0XRCFR|          4|           16|         21|   Y|                N|
| RXYYJ8SGI9IX6|          5|           36|         41|   Y|                N|
|R1SAUBFC7T6JHC|          5|           73|         75|   Y|                N|
|R329HVJKKHWSG3|          5|           59|         59|   Y|                N|
| RZL21K50M52KW|          5|           22|         22|   Y|                N|
|R24RC5J2D92SBE|          5|           19|         20|   Y|                N|
|R1803HITBY7Y08|          4|           92|        104|   Y|                N|
|R1888B8Z6QWYZ4|          4|           18|         25|   Y|                N|
|R1S538NK2I5PRL|          5|           21|         22|   Y|     

In [47]:
# Filter hv table for Vine program (unpaid)
vp_no = hv.filter(hv.vine == 'N')
vp_no.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|     

In [68]:
# Total number of reviews paid
total_reviews_yes = vp_yes.count()
# Total number of 5 star ratings paid
five_star_yes = vp_yes.filter('star_rating == 5').count()
# Percentage of 5 star reviews for paid ratings
percent_paid = round((five_star_yes/total_reviews_yes)*100,2)

# Total number of reviews unpaid
total_reviews_no = vp_no.count()
# Total number of 5 star ratings unpaid
five_star_no = vp_no.filter('star_rating == 5').count()
# Percentage of 5 sar reviews for unpaid ratings
percent_unpaid = round((five_star_no/total_reviews_no)*100,2)

# Print statement
print(f'Total number of ratings paid: {total_reviews_yes}')
print(f'Total number of 5 star ratings paid: {five_star_yes}')
print(f'Percentage of 5 star ratings for paid: {percent_paid}%')
print(f'-----------------------------------------------------')
print(f'Total number of ratings unpaid: {total_reviews_no}')
print(f'Total number of 5 star ratings paid: {five_star_no}')
print(f'Percentage of 5 star ratings for unpaid: {percent_unpaid}%')
print(f'-----------------------------------------------------')

Total number of ratings paid: 334
Total number of 5 star ratings paid: 139
Percentage of 5 star ratings for paid: 41.62%
-----------------------------------------------------
Total number of ratings unpaid: 61614
Total number of 5 star ratings paid: 32665
Percentage of 5 star ratings for unpaid: 53.02%
-----------------------------------------------------
